# Reflection and Blogpost Writing

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
llm_config = {"model": "gpt-3.5-turbo","api_key":OPENAI_API_KEY}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

In [4]:
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task,"role": "user"}])

In [6]:
print(reply)

Title: Unveiling the Power of Hope AI

Step into the revolutionary world of Hope AI, where technology meets compassion to make a lasting impact. With its advanced algorithms, Hope AI is transforming the way we address mental health challenges, providing support and resources 24/7. By combining innovation with empathy, this AI-powered platform offers a ray of hope to those in need. Join the movement and discover a new way to prioritize mental well-being. Hope AI is not just a tool; it's a beacon of light guiding us towards a brighter, more resilient future.

---
Refined version:

Title: Exploring the Transformational Impact of Hope AI

Discover the transformative potential of Hope AI, where cutting-edge technology intersects with compassion to revolutionize mental health support. Through its sophisticated algorithms, Hope AI offers round-the-clock assistance, reshaping the narrative around mental well-being. By harmonizing innovation with empathy, this pioneering platform provides a lif

## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [11]:
critic = autogen.AssistantAgent(##
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [12]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Embracing the Future: Hope AI"

In a world filled with uncertainty, Hope AI emerges as a shining beacon of possibility. This innovative technology is designed to empower individuals, offering solutions to complex problems and a glimpse into a brighter future. With its ability to analyze data, predict outcomes, and streamline processes, Hope AI has the potential to revolutionize various industries and transform the way we live and work. As we navigate challenges and embrace change, Hope AI serves as a constant reminder that the future is full of promise and potential. So let's welcome Hope AI with open arms and unlock the endless possibilities it holds.

------------------------------------------------------------------------------

## Nested chat

In [13]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [14]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [15]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [17]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [18]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [19]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [20]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling Hope AI: The Future of Mental Health Support

Step into a world where technology meets empathy – introducing Hope AI. This groundbreaking innovation offers a virtual helping hand for those struggling with mental health concerns. With 24/7 accessibility and personalized assistance, Hope AI aims to provide support and resources to promote emotional well-being. Through a combination of artificial intelligence and compassion, users can feel heard and understood. Whether you're feeling anxious, lonely, or lost, Hope AI is here to guide you towards a brighter tomorrow. Embrace the future of mental health care with Hope AI by your side.

--------------------------------------------------------------------------------

**********

In [21]:
print(res.summary)

Title: Embracing Tomorrow: Hope AI Revolutionizing Mental Health Support

Dive into the future of mental health care with Hope AI. This innovative virtual support system combines technology with empathy to offer personalized assistance 24/7. Understanding and guiding those facing challenges, Hope AI stands out in providing comfort and resources. From anxiety to loneliness, users find solace in a brighter future. Take the step towards improved well-being and embrace the transformative power of Hope AI. Join the movement towards a more supportive and understanding mental health landscape.
